In [6]:
import cvxpy as cp
import numpy as np
import csv

In [7]:
#import the resource codes
def load_file_60d(filename):
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        data={}
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break
            if next_line[1]!='1':
                break
            data[next_line[2]] = 'UNK'
#             data.append({next_line[2]:'UNK'})
    return data  

In [8]:
#map resource code to resource type
def code_to_type(filename, data):
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break
            if next_line[2] in data:
                if data[next_line[2]]!='UNK':
                    break
                data[next_line[2]] = next_line[3]

In [9]:
#add the resource types together
def wind_solar_added(filename, data, wind, solar, percent_wind , percent_solar):
    wind_total = 20271
    solar_total = 1008
    with open(filename) as file:
            reader = csv.reader(file)
            next(reader)
            while True:
                try:
                    next_line = next(reader)
                except StopIteration:
                    break
                if next_line[0] not in (wind and solar):
                    solar[next_line[0]] = 0
                    wind[next_line[0]] = 0
                    percent_wind[next_line[0]] = 0
                if (data[next_line[2]])=='WIND':
                    wind[next_line[0]] = (float(wind[next_line[0]]) + float(next_line[3]))
                    percent_wind[next_line[0]] = (float(wind[next_line[0]]) + float(next_line[3]))/wind_total
                if (data[next_line[2]])=='PVGR':
                    solar[next_line[0]]= float(solar[next_line[0]]) + float(next_line[3])
                    percent_solar[next_line[0]] = (float(solar[next_line[0]]) + float(next_line[3]))/solar_total
#     print (solar)

In [10]:
#get the total amount of solar and wind per region
def map_to_regions(filename, solar_data, wind_data, wind_regions, solar_regions):
    jan_march_wind=0
    apr_sep_wind=0
    oct_dec_wind=0
    jan_march_solar=0
    apr_sep_solar=0
    oct_dec_solar = 0

    
    with open(filename) as file:
            reader = csv.reader(file)
            next(reader)
            while True:
                try:
                    next_line = next(reader)
                except StopIteration:
                    break
                if (next_line[0] == 'WIND-C') or (next_line[0] == 'WIND'):
                    jan_march_wind+= float(next_line[2])
                    apr_sep_wind+= float(next_line[3])
                    oct_dec_wind+= float(next_line[4])
                    wind_regions[next_line[1]] += float(next_line[3])
                if next_line[0] == 'SOLAR':
                    jan_march_solar+= float(next_line[2])
                    apr_sep_solar+= float(next_line[3])
                    oct_dec_solar+= float(next_line[4])
                    solar_regions[next_line[1]] += float(next_line[3])

    for i in wind_regions:
        wind_regions[i] = float(wind_regions[i]/apr_sep_wind)
    for i in solar_regions:
        solar_regions[i] = float(solar_regions[i]/apr_sep_solar)
    
    for i in wind_data:
        wind_regions_copy = dict(wind_regions)
        for j in wind_regions_copy:
            wind_regions_copy[j] = float(wind_regions_copy[j]*wind_data[i])
        wind_data[i] = wind_regions_copy
            
    for i in solar_data:
        solar_regions_copy = dict(solar_regions)
        for j in solar_regions_copy:
            solar_regions_copy[j] = float(solar_regions_copy[j]*solar_data[i])
        solar_data[i] = solar_regions_copy
        

In [40]:
def write_to_csv(wind_data, solar_data):
    with open('wind_per_15min.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                                quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['Date','Time','Coast', 'East', 'FarWest', 'North', 'NorthCentral', 'Northwest', 'South', 
                    'SouthCentral','West'])
        count = 4
        for i in wind_data:
            if count == 4:
                count = 0
                split_string = i.split()
                split_string[1] = split_string[1][:2]
                temp = split_string
                for j in wind_data[i]:
                    temp.append(str(wind_data[i][j]))
                writer.writerow(temp)
                print(temp)
            count+=1

    
    with open('solar_per_15min.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                                quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['Time','Coast', 'East', 'FarWest', 'North', 'NorthCentral', 'Northwest', 'South', 
                    'SouthCentral','West'])
        count = 4
        for i in solar_data:
            if count == 4:
                count = 0
                temp = [i];
                for j in solar_data[i]:
                    temp.append(str(solar_data[i][j]))
                writer.writerow(temp)
            count+=1

In [41]:
data = load_file_60d('60d_SCED_SMNE_GEN_RES-17-APR-18.csv')
code_to_type('60d_SCED_Gen_Resource_Data-17-APR-18.csv', data)

wind = {}
solar = {}
percent_wind = {}
percent_solar = {}
wind_solar_added('60d_SCED_SMNE_GEN_RES-17-APR-18.csv',data, wind, solar, percent_wind, percent_solar)


wind_regions = {'Coast':0, 'East':0, 'FarWest':0, 'North':0, 'NorthCentral':0, 'Northwest':0, 'South':0, 
                    'SouthCentral':0,'West':0}
solar_regions= {'Coast':0, 'East':0, 'FarWest':0, 'North':0, 'NorthCentral':0, 'Northwest':0, 'South':0, 
                    'SouthCentral':0,'West':0}
map_to_regions('zonal_generator_capacities_wo_biomass.csv',solar, wind, wind_regions, solar_regions)

write_to_csv(wind, solar)


['02/16/2018', '0', '0.0', '0.0', '459.91305666110424', '442.5268610173908', '230.89302130433526', '447.3043324198788', '531.7787131275077', '0.0', '638.8510764633824']
['02/16/2018', '0', '0.0', '0.0', '418.05325280398375', '402.2494926422089', '209.87788280414838', '406.5921339981377', '483.3779288825146', '0.0', '580.7049108624068']
['02/16/2018', '0', '0.0', '0.0', '388.5032486017616', '373.81657382578163', '195.0427098274558', '377.8522565120774', '449.2104640106717', '0.0', '539.6579092157504']
['02/16/2018', '0', '0.0', '0.0', '389.43858618513474', '374.7165526343126', '195.51228319015263', '378.7619513930085', '450.2919567172214', '0.0', '540.95715787447']
['02/16/2018', '0', '0.0', '0.0', '451.86277138769157', '434.7809024699192', '226.8514863615478', '439.4747493544281', '522.4704965396077', '0.0', '627.6686728802055']
['02/16/2018', '0', '0.0', '0.0', '529.7496493608418', '509.7234054597842', '265.95352165869804', '515.2263233779515', '612.5279174764547', '0.0', '735.8589386